#### Notes on current version:
recentered_input vs norm_at_center?

# NEU (Reconfigurations Map and Related Functions)

### Basic Algorithm (NEU-OLS)

1. Perform Basic Algorithm (in this case OLS)
2. Map predictions to their graph; ie $x\mapsto (x,\hat{f}_{OLS}(x))$ where $\hat{f}_{OLS}$ is the least-squares regression function.

## Initializations:

In [1]:
# Deep Learning & ML
import tensorflow as tf
import tensorflow_probability as tfp
import keras as K
import keras.backend as Kb
from keras.layers import *

from keras.models import Model
from keras.models import Sequential
from keras import layers
from keras import utils as np_utils
from scipy import linalg as scila

from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.constraints import NonNeg



# Linear Regression
from sklearn.linear_model import LinearRegression

# General
import numpy as np
import time

# Alerts
import os as beepsnd

# Others
import math

# General Outputs
print('TensorFlow:', tf.__version__)

Using TensorFlow backend.


TensorFlow: 2.1.0


In [2]:
N_Reconfigurations = 20
d = 1 # Dimension of X
D = 1 # Dimension of Y


# Data Meta-Parameters
noise_level = 0.1
uncertainty_level= 0.9

# Training meta-parameters
Pre_Epochs = 10
Full_Epochs = 100

# Height Per Reconfiguration
Height_factor_Per_reconfig = 2

# Number of Datapoints
N_data = 10**3
# Unknown Function
def unknown_f(x):
    return np.sin(x) #+ (x % 2)

# Generate Data
%run Data_Generator.ipynb

<Figure size 640x480 with 1 Axes>

#### Prepare data for NEU

In [3]:
# Reshape Data Into Compatible Shape
data_x = np.array(data_x).reshape(-1,d)
data_y = np.array(data_y)
# Perform OLS Regression
linear_model = LinearRegression()
reg = linear_model.fit(data_x, data_y)
model_pred_y = linear_model.predict(data_x)
# Map to Graph
data_NEU = np.concatenate((data_x,model_pred_y.reshape(-1,D)),1)
NEU_targets  = data_y.reshape(-1,D)

## Helper Functions:

In [4]:
# # Bump Function
# def bump_function(self, x):
#         return tf.math.exp(-self.sigma / (self.sigma - tf.math.pow(x, 2)))

### Build Reconfiguration Unit
$$
x \mapsto \exp\left(
\psi(x-c;\sigma)\operatorname{Skew}_d\left(
    F(\|x\|)
\right)
\right) (x-c) + c
$$
where:
#### Workflow
1. Shifts $x \in \mathbb{R}^d$ to $x- c$; c trainable.
2. Applies the map $\psi(x;\sigma)\triangleq e^{\frac{\sigma}{\sigma-|x|}}I_{\{|x|<\sigma\}}$ component-wise.  
3. Applies transformation $x \mapsto x +b$, $b \in \mathbb{R}^d$ trainable.
4. Applies the diagonalization map to that output: $ \left(x_1,\dots,x_d\right)\mapsto
                \begin{pmatrix}
                x_1 & & 0\\
                &\ddots &\\
                0 & & x_d\\
                \end{pmatrix}.$
5. Applies map $X \mapsto XA$, $A$ is a trainable $d\times d$ matrix.
6. Applies matrix exponential.
7. Multiplies output with result of (1).
8. Re-centers output to $x +c$ where $c$ is as in (1).

### Helper Function: Build and Training NEU Units (Core)

In [5]:
class Reconfiguration_unit_steps(tf.keras.layers.Layer):

    def __init__(self, *args, **kwargs):
        super(Reconfiguration_unit_steps, self).__init__(*args, **kwargs)

    def build(self, input_shape):
        self.location_in = self.add_weight(name='location_in',
                                     shape=input_shape[1:],
                                     initializer='GlorotUniform',
                                     trainable=True)
        self.location_out = self.add_weight(name='location_out',
                                     shape=input_shape[1:],
                                     initializer='GlorotUniform',
                                     trainable=True)

###########################################################################################
        # Tangential ffNN
        self.tangential_ffNN_W1 = self.add_weight(name='TWeights1 ',
                                    shape=(input_shape[1], input_shape[1]),
                                    initializer='GlorotUniform',
                                    trainable=True)
        
        self.tangential_ffNN_b1 = self.add_weight(name='TBiases1',
                                    shape=(input_shape[1], input_shape[1]),
                                    initializer='GlorotUniform',#Previously 'ones'
                                    trainable=True)
        
        self.tangential_ffNN_W2 = self.add_weight(name='TWeights2 ',
                                    shape=(input_shape[1], input_shape[1]),
                                    initializer='GlorotUniform',
                                    trainable=True)

        self.tangential_ffNN_b2 = self.add_weight(name='TBiases2',
                                    shape=(input_shape[1], input_shape[1]),
                                    initializer='ones',#Previously 'ones'
                                    trainable=True)
###########################################################################################
        # Bump Function
        self.sigma = self.add_weight(
                        name='sigma',
                        shape=[1],
                        initializer=RandomUniform(minval=.95, maxval=1.05),
                        trainable=True,
                        constraint=tf.keras.constraints.NonNeg()
                        )
        super().build(input_shape)
    
    def bump_function(self, x):
        return tf.math.exp(-self.sigma / (self.sigma - tf.math.pow(x, 2)))
    
    def call(self, input):
        # Input Processing
        #--------------------#
        recentered_input = input + self.location_in
        norm_at_recenter = tf.math.sqrt(tf.math.reduce_sum((recentered_input*recentered_input)))
        
        # Tangential ffNN: so_{d\times }
        #------------------------#
        # Tangential ffNN: Build
        T_ffNN = (norm_at_recenter*self.tangential_ffNN_W1) + self.tangential_ffNN_b1
        #T_ffNN = tf.linalg.matvec(self.tangential_ffNN_W1,recentered_input) + self.tangential_ffNN_b1
        T_ffNN = tf.nn.relu(T_ffNN)
        T_ffNN = tf.linalg.matmul(self.tangential_ffNN_W2,T_ffNN) + self.tangential_ffNN_b2

        # Anti-Symmetrize
        #T_ffNN = tf.linalg.diag(T_ffNN)
        T_ffNN_so = .5*(T_ffNN - tf.transpose(T_ffNN))
        #x_out = T_ffNN
        
        # Apply Bump Function
        #------------------------------#
        # Logic: When to bump or not to bump
        greater = tf.math.greater(norm_at_recenter, -1)
        less = tf.math.less(norm_at_recenter, 1)
        condition = tf.logical_and(greater, less)

        bumper = tf.where(
            condition, 
            self.bump_function(tf.where(condition, norm_at_recenter, 0.0)),
            0.0)
       
        T_ffNN_so = bumper*T_ffNN_so 
        # Little Bump to smooth numerical issues
        x_out = T_ffNN_so 
        
        
        
        # Map so_d ->> SO_d
        #------------------------------#
        # **Note/fact:** TF uses Sylvester's method to for (fast) computation but this requires additional assumptions on the matrix which are typically not satified...
        # **Instead:** Use a truncated power series representation (standard definition of expm) of order 4
        x_out = tf.linalg.diag(tf.ones(d+D)) + x_out + tf.linalg.matmul(x_out,x_out)/2 + tf.linalg.matmul(x_out,tf.linalg.matmul(x_out,x_out))/6 +tf.linalg.matmul(x_out,tf.linalg.matmul(x_out,tf.linalg.matmul(x_out,x_out)))/24 #+tf.linalg.matmul(x_out,tf.linalg.matmul(x_out,tf.linalg.matmul(x_out,tf.linalg.matmul(x_out,x_out))))/120
        #x_out = tf.linalg.expm(T_ffNN_so)
        
        # 8. Muliply by output of (1)
        x_out = tf.linalg.matvec(x_out,recentered_input)
        
        # 9. Recenter Transformed Data
        x_out = x_out + self.location_out
        
        # Return Output
        return x_out

### Helper Function: Projection Layer (For Regression)
Maps $\mathbb{X}\left((x,f(x))\mid \theta \right) \in \mathbb{R}^{d\times D}$ to an element of $\mathbb{R}^D$ by post-composing with the second canonical projection
$$
(x_1,x_2)\mapsto x_2
,
$$
where $x_1 \in \mathbb{R}^d$ and $x_2 \in \mathbb{R}^D$.  

In [6]:
projection_layer = tf.keras.layers.Lambda(lambda x: x[:, -D:])

## Robust Loss Function
This loss function prevents overfitting... it is especially userful for greedy approaches to training...like we use...

In [7]:
def above_percentile(x, p): #assuming the input is flattened: (n,)

    samples = Kb.cast(Kb.shape(x)[0], Kb.floatx()) #batch size
    p =  (100. - p)/100.  #100% will return 0 elements, 0% will return all elements

    #samples to get:
    samples = Kb.cast(tf.math.floor(p * samples), 'int32')
        #you can choose tf.math.ceil above, it depends on whether you want to
        #include or exclude one element. Suppose you you want 33% top,
        #but it's only possible to get exactly 30% or 40% top:
        #floor will get 30% top and ceil will get 40% top.
        #(exact matches included in both cases)

    #selected samples
    values, indices = tf.math.top_k(x, samples)

    return values

def Robust_MSE(p):
    def loss(y_true, y_predicted):
        ses = Kb.pow(y_true-y_predicted,2)
        above = above_percentile(Kb.flatten(ses), p)
        return Kb.mean(above)
    return loss

### Helper Functions: Compiling and Training NEU-OLS

#### First Unit
These are helper functions for training the reconfiguration map.

Build and greedily-initialize the first reconfiguration unit.

In [8]:
# define and fit the base model
def get_base_model(trainx, trainy, Pre_Epochs_in):
    # Define Model
    #----------------#
    # Initialize
    input_layer = tf.keras.Input(shape=[d+D])
    # Apply Reconfiguration Unit
    reconfigure  = Reconfiguration_unit_steps()
    current_layer = reconfigure(input_layer)
    # Output
    output_layer = projection_layer(current_layer)
    reconfiguration_basic = tf.keras.Model(inputs=[input_layer], outputs=[output_layer])
    
    # Compile Model
    #----------------#
    # Define Optimizer
    optimizer_on = tf.keras.optimizers.SGD(learning_rate=10**(-2), momentum=0.01, nesterov=True)
    # Compile
    reconfiguration_basic.compile(loss = Robust_MSE(uncertainty_level),
                    optimizer = optimizer_on,
                    metrics = ['mse'])
    
    # Fit Model
    #----------------#
    reconfiguration_basic.fit(trainx, trainy, epochs=Pre_Epochs_in, verbose=0)
        
    # Return Output
    return reconfiguration_basic

#### Greedy Initialization of Subsequent Units
Build reconfiguration and pre-train using greedy approach.

In [9]:
def add_reconfiguration_unit_greedily(model, trainx, trainy, Pre_Epochs_in):

    # Dissasemble Network
    layers = [l for l in model.layers]

    # Define new reconfiguration unit to be added
    new_reconfiguration_unit  = Reconfiguration_unit_steps()
    current_layer_new = new_reconfiguration_unit(layers[len(layers)-2].output)

    # Output Layer
    output_layer_new = projection_layer(current_layer_new)

    for i in range(len(layers)):
        layers[i].trainable = False


    # build model
    new_model = tf.keras.Model(inputs=[layers[0].input], outputs=output_layer_new)
    #new_model.summary()


    # Compile new Model
    #-------------------#
    # Define Optimizer
    optimizer_on = tf.keras.optimizers.SGD(learning_rate=10**(-2), momentum=0.01, nesterov=True)
    # Compile Model
    new_model.compile(loss = Robust_MSE(uncertainty_level),
                    optimizer = optimizer_on,
                    metrics = ['mse'])

    # Fit Model
    #----------------#
    new_model.fit(trainx, trainy, epochs=Pre_Epochs_in, verbose=0)

    # Return Output
    return new_model

#### Train and Compile (entire) reconfiguration using greedy-initializations past from previous helper functions.
Train reconfiguration together (initialized by greedy) layer-wise initializations.

In [10]:
def build_reconfiguration(model_greedy_initialized, trainx, trainy, Full_Epochs_in):

    # Dissasemble Network
    layers = [l for l in model_greedy_initialized.layers]

    # Define new reconfiguration unit to be added
    new_reconfiguration_unit  = Reconfiguration_unit_steps()
    current_layer_new = new_reconfiguration_unit(layers[len(layers)-2].output)

    # Output Layer
    output_layer_new = projection_layer(current_layer_new)

    for i in range(len(layers)):
        layers[i].trainable = True


    # build model
    reconfiguration = tf.keras.Model(inputs=[layers[0].input], outputs=output_layer_new)
    #new_model.summary()



    # Compile new Model
    #-------------------#
    # Define Optimizer
    optimizer_on = tf.keras.optimizers.SGD(learning_rate=10**(-5), momentum=0.01, nesterov=True)
    #optimizer_on = tf.keras.optimizers.Adagrad(learning_rate=10**(-5), initial_accumulator_value=0.1, epsilon=1e-07,name='Adagrad')

    # Compile Model
    reconfiguration.compile(loss = Robust_MSE(uncertainty_level),
                    optimizer = optimizer_on,
                    metrics = ['mse'])

    # Fit Model
    #----------------#
    reconfiguration.fit(trainx, trainy, epochs=Full_Epochs_in, verbose=1)

    # Return Output
    return reconfiguration

## Train NEU-OLS

In [ ]:
# Base Model
model = get_base_model(data_NEU,NEU_targets,Pre_Epochs)

# Greedy Initialization
NEU_OLS_Greedy_init = get_base_model(data_NEU,NEU_targets,Pre_Epochs)
for i in range(N_Reconfigurations):
    # Update Model
    NEU_OLS_Greedy_init_temp = add_reconfiguration_unit_greedily(NEU_OLS_Greedy_init,data_NEU,NEU_targets,Pre_Epochs)
    
    # Check for Blowup
    if math.isnan(np.mean(NEU_OLS_Greedy_init.predict(data_NEU))):
        NEU_OLS_Greedy_init = NEU_OLS_Greedy_init
        break
    else: #Update Model if not explosion
        NEU_OLS_Greedy_init = NEU_OLS_Greedy_init_temp
    
    print(np.mean((NEU_OLS_Greedy_init.predict(data_NEU) - data_y)**2))
    
    # Update User on Status of Initialization
    print(((i+1)/N_Reconfigurations))

0.1717893805810051
0.05
0.17168614623215142
0.1
0.17167537757360082
0.15
0.17168669466761646
0.2
0.17167671198938977
0.25
0.17203800039631806
0.3
0.1718904351093933
0.35
0.17167190919912467
0.4
0.1716746947196948
0.45
0.1716741851421072
0.5


In [ ]:
# # Train Full Model Using Initializatoins
NEU_OLS = NEU_OLS_Greedy_init
NEU_OLS = build_reconfiguration(NEU_OLS,data_NEU,NEU_targets,Full_Epochs)

## Make Predictions

In [ ]:
# # Predictions (for comparison: TEMP)
NEU_OLS_prediction = NEU_OLS(data_NEU)
NEU_OLS_single_unit_prediction = model.predict(data_NEU)
NEU_OLS_greedy_initializations = NEU_OLS_Greedy_init.predict(data_NEU)

# Visualize Predictions

In [ ]:
# Adjust Figure Details
plt.figure(num=None, figsize=(12, 10), dpi=80, facecolor='w', edgecolor='k')

# Data Plot
plt.plot(data_x,true_y,color='k',label='true',linestyle='--')

# Plot Models
plt.plot(data_x,model_pred_y,color='r',label='OLS')
plt.plot(data_x,NEU_OLS_single_unit_prediction,color='b',label='NEU_Unit')
plt.plot(data_x,NEU_OLS_greedy_initializations,color='g',label='NEU_Greedy_Init')
plt.plot(data_x,NEU_OLS_prediction,color='Aqua',label='NEU-OLS')

#### The END